<a href="https://colab.research.google.com/github/omkar-salunke/Trading_algos/blob/main/EMA_%3E_SMA_crossover_strategyipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Importing data 
!git clone https://github.com/omkar-salunke/Trading_algos.git

import zipfile
with zipfile.ZipFile("/content/Trading_algos/Data_NSE.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/Trading_algos")
    

fatal: destination path 'Trading_algos' already exists and is not an empty directory.


In [3]:
!pip install ta
# Install backtesting infrastructure
# https://pypi.org/project/Backtesting/
!pip install backtesting
!apt-get install nodejs -y
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for ta: filename=ta-0.10.2-py3-none-any.whl size=29104 sha256=83fe5f95972753c44186c589bc6dad4cbcf2e220792bd1b1157d8b155cb11933
  Stored in directory: /root/.cache/pip/wheels/31/31/f1/f2ff471bbc5b84a4b973698ceecdd453ae043971791adc3431
Successfully built ta
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 175 kB 7.6 MB/s 
  Created wheel for backtesting: filename=Backtesting-0.3.3-py3-none-any.whl size=173832 sha256=a7a4f33ab7f85aa95d421589b7388e41e161918690d0daf0cdf3e708586e3897
  Stored in directory: /root/.cache/pip/wheels/48/1d/ea/65dacebc37da7655d8a1fd0f315ac39d102e31d6545237a1c6
Successfully built backtesting
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
 

In [17]:
import os
import glob

path = '/content/Trading_algos'
extension = 'csv'
os.chdir(path)
companies_list = glob.glob('*.{}'.format(extension))
companies_list

['MM.csv',
 'SHREECEM.csv',
 'NTPC.csv',
 'NESTLEIND.csv',
 'HCLTECH.csv',
 'BAJAJ-AUTO.csv',
 'BAJAJFINSV.csv',
 'BRITANNIA.csv',
 'ONGC.csv',
 'BHARTIARTL.csv',
 'JSWSTEEL.csv',
 'HINDUNILVR.csv',
 'MARUTI.csv',
 'KOTAKBANK.csv',
 'ULTRACEMCO.csv',
 'GRASIM.csv',
 'BPCL.csv',
 'SBIN.csv',
 'ICICIBANK.csv',
 'HINDALCO.csv',
 'BAJFINANCE.csv',
 'ASIANPAINT.csv',
 'IOC.csv',
 'TECHM.csv',
 'TATAMOTORS.csv',
 'GAIL.csv',
 'WIPRO.csv',
 'COALINDIA.csv',
 'POWERGRID.csv',
 'NIFTY50_all.csv',
 'EICHERMOT.csv',
 'ZEEL.csv',
 'stock_metadata.csv',
 'INFY.csv',
 'UPL.csv',
 'DRREDDY.csv',
 'ADANIPORTS.csv',
 'ITC.csv',
 'AXISBANK.csv',
 'VEDL.csv',
 'SUNPHARMA.csv',
 'HDFC.csv',
 'CIPLA.csv',
 'TITAN.csv',
 'INFRATEL.csv',
 'HEROMOTOCO.csv',
 'INDUSINDBK.csv',
 'LT.csv',
 'HDFCBANK.csv',
 'TATASTEEL.csv',
 'TCS.csv',
 'RELIANCE.csv']

In [18]:
stock = "BAJAJ-AUTO.csv"
import backtesting

from backtesting import Backtest, Strategy
from backtesting.lib import crossover

import pandas as pd
import ta.trend as tr
for stock in companies_list:
  try:
    # Preprocess Open, High, Low, Close (OHLC) data for Nifty index from 2009 to 2020
    stock_data = pd.read_csv('/content/Trading_algos/'+stock);
    stock_data.drop(columns=['Series','Symbol','Prev Close', 'VWAP', 'Turnover', 'Trades', 'Deliverable Volume', '%Deliverble'], inplace=True)
    stock_data.Date= pd.to_datetime(stock_data.Date,format='%Y-%m-%d')
    stock_data.set_index('Date',drop=True, inplace=True)


    # Backtest a simple EMA -> SMA crossover strategy on above data
    backtesting.set_bokeh_output(notebook=True)



    # BOKEH_RESOURCES=inline
    class EmaSmaCross(Strategy):
        def init(self):
          
          cp = pd.Series(self.data.Close)
          
          self.ema1 = self.I(tr.ema_indicator,close=cp, window=20);
          self.ema2 = self.I(tr.sma_indicator,close=cp, window=50);
          self.ema3 = self.I(tr.sma_indicator,close=cp, window=200)
            

        def next(self):
          # Buy or sell on ema -> sma cross over
          # Have a 200 pts stop loss for long trades and 130 points for short trades
          if crossover(self.ema1, self.ema2):
            self.buy(sl=self.data.High-200)
          elif crossover(self.ema2, self.ema1):
            self.sell(sl=self.data.Low+130)
          elif crossover(self.ema2, self.ema3):
            self.sell(sl=self.data.Low+100)     

    bt = Backtest(stock_data, EmaSmaCross, commission=.002,cash=1000000,
                  exclusive_orders=True)
    # Run the backtest
    stats = bt.run()
    
    # Print the performance metrics for this strategy 
    print(stock+ " Return %: "+str(stats['Return [%]']))
  except:
    print(stock)

MM.csv
SHREECEM.csv
NTPC.csv
NESTLEIND.csv
HCLTECH.csv
BAJAJ-AUTO.csv Return %: -44.494028889999946
BAJAJFINSV.csv
BRITANNIA.csv Return %: 79.62737899999959
ONGC.csv
BHARTIARTL.csv
JSWSTEEL.csv
HINDUNILVR.csv
MARUTI.csv Return %: 51.76938042999998
KOTAKBANK.csv
ULTRACEMCO.csv
GRASIM.csv
BPCL.csv
SBIN.csv
ICICIBANK.csv
HINDALCO.csv
BAJFINANCE.csv
ASIANPAINT.csv Return %: -94.5372250300001
IOC.csv
TECHM.csv Return %: -36.83448518999984
TATAMOTORS.csv
GAIL.csv
WIPRO.csv
COALINDIA.csv
POWERGRID.csv


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:45: UserWarning: Data index is not sorted in ascending order. Sorting.


NIFTY50_all.csv Return %: -100.0
EICHERMOT.csv
ZEEL.csv
stock_metadata.csv
INFY.csv
UPL.csv
DRREDDY.csv
ADANIPORTS.csv
ITC.csv
AXISBANK.csv
VEDL.csv
SUNPHARMA.csv Return %: -99.45199640999999
HDFC.csv Return %: -97.38334727999994
CIPLA.csv
TITAN.csv
INFRATEL.csv
HEROMOTOCO.csv
INDUSINDBK.csv
LT.csv
HDFCBANK.csv Return %: -71.26996960999999
TATASTEEL.csv
TCS.csv Return %: -98.53138967999998
RELIANCE.csv


In [21]:
stock = "BRITANNIA.csv"
# Preprocess Open, High, Low, Close (OHLC) data for Nifty index from 2009 to 2020
stock_data = pd.read_csv('/content/Trading_algos/'+stock);
stock_data.drop(columns=['Series','Symbol','Prev Close', 'VWAP', 'Turnover', 'Trades', 'Deliverable Volume', '%Deliverble'], inplace=True)
stock_data.Date= pd.to_datetime(stock_data.Date,format='%Y-%m-%d')
stock_data.set_index('Date',drop=True, inplace=True)


# Backtest a simple EMA -> SMA crossover strategy on above data
backtesting.set_bokeh_output(notebook=True)



# BOKEH_RESOURCES=inline
class EmaSmaCross(Strategy):
    def init(self):
      
      cp = pd.Series(self.data.Close)
      
      self.ema1 = self.I(tr.ema_indicator,close=cp, window=20);
      self.ema2 = self.I(tr.sma_indicator,close=cp, window=50);
      self.ema3 = self.I(tr.sma_indicator,close=cp, window=200)
        

    def next(self):
      # Buy or sell on ema -> sma cross over
      # Have a 200 pts stop loss for long trades and 130 points for short trades
      if crossover(self.ema1, self.ema2):
        self.buy(sl=self.data.High-200)
      elif crossover(self.ema2, self.ema1):
        self.sell(sl=self.data.Low+130)
      elif crossover(self.ema2, self.ema3):
        self.sell(sl=self.data.Low+100)     

bt = Backtest(stock_data, EmaSmaCross, commission=.002,cash=1000000,
              exclusive_orders=True)
# Run the backtest
stats = bt.run()

# Print the performance metrics for this strategy 
print(stats)

Start                     2000-01-03 00:00:00
End                       2021-04-30 00:00:00
Duration                   7788 days 00:00:00
Exposure Time [%]                   72.497644
Equity Final [$]                   1796273.79
Equity Peak [$]                  5765308.4974
Return [%]                          79.627379
Buy & Hold Return [%]              355.674462
Return (Ann.) [%]                    2.821348
Volatility (Ann.) [%]               24.745189
Sharpe Ratio                         0.114016
Sortino Ratio                        0.156788
Calmar Ratio                         0.040982
Max. Drawdown [%]                  -68.843406
Avg. Drawdown [%]                   -5.365157
Max. Drawdown Duration     3172 days 00:00:00
Avg. Drawdown Duration      104 days 00:00:00
# Trades                                  114
Win Rate [%]                        29.824561
Best Trade [%]                      223.75624
Worst Trade [%]                    -49.069712
Avg. Trade [%]                    

In [22]:
# Plot the price movements, trades and other meta-data such as drawdown etc.
bt.plot(resample=True);